<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/3_1_scrapyRSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3 - Scrapy
En esta sesión se va a ver una breve introducción a cómo se pueden crear Crawlers sencillos con la librería Scrapy de cara a hacer la práctica 1.

Lo primero que haremos será instalarnos la librería

In [ ]:
!pip3 install -U scrapy

## Apartado 1.1 Crawler de ficheros RSS
Para desarrollar un Crawler en Scrapy debemos definir una clase Spider que herede de scrapy.Spider. Cada clase definirá los métodos necesarios para realizar el scrapping del sitio web y cómo obtener información estructurada.

Para poder crear Spiders adecuados es NECESARIO conocerse los selectores CSS de HTML.
https://www.w3schools.com/cssref/css_selectors.asp

En el ejemplo de abajo está comentado cómo se haría el scrapping the un fichero en RSS.

In [1]:
import scrapy
import sys
import json
import locale
import time
import string
import random
from bs4 import BeautifulSoup

class RSSSpider (scrapy.Spider):

    # Es obligatorio poner el nombre del Spider
    name = 'RSS'

    # Estas son las URLs donde empieza a buscar y se podrían poner varias distintas de RSS feeds
    start_urls = ['https://www.abc.es/rss/2.0/portada/',
                  'https://www.laverdad.es/rss/2.0/portada',
                  'https://www.europapress.es/rss/rss.aspx',
                  'https://feeds.elpais.com/mrss-s/pages/ep/site/elpais.com/portada']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }


    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        #Buscamos todos los elementos en el archivo XML con la etiqueta <item>
        for item in response.css ('item'):
            #Obtenemos por cada elemento <item> el texto del subelemento <link>
            url = str (item.css ('link::text').get ()).strip()
            #Obtenemos por cada elemento <item> el texto del subelemento <title>. Además co el BeautifulSoup
            #procesamos el texto en html y nos quedamos con el texto
            title = BeautifulSoup(str(item.css ('title').get()), 'html.parser').get_text().strip()
            #Obtenemos por cada elemento <item> el texto del subelemento <description>
            content = BeautifulSoup(str(item.css ('description').get()), 'html.parser').get_text().strip()
            #Extraemos la fecha
            date = BeautifulSoup(str(item.css ('pubDate').get()), 'html.parser').get_text().strip()

            #Imprimimos la información obtenida para comprobar lo que estamos extrayendo
            print ("-------------------------")
            print ('URL:' + url)
            print ('Título:' + title)
            print ('Descripción:' + content)
            print ('Fecha:' + date)
            print ("-------------------------")

            data = {
                'url' : url,
                'title': title,
                'content': content,
                'date': date
            }

            #Creamos para cada item un fichero json y para ello obtenemos un número aleatorio.
            filename = str(random.random()).replace(".","") + ".json"

            # Si tenemos descripción, url y título entonces lo guardamos a disco en la carpeta 'rss'
            if content and title and url:
                with open ('rss/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)

## Apartado 1.2 - Lanzar el Crawler RSSSpyder
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

In [2]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('rss')):
    os.mkdir('rss')

# Creamos el proceso con el RSSSpider
process.crawl(RSSSpider)
# Ejecutamos el Crawler
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.3 19 Sep 2023), cryptography 41.0.4, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
DEBUG:scrap

-------------------------
URL:https://www.abc.es/deportes/futbol/tere-abelleira-fin-hablamos-futbol-20231026181641-nt.html
Título:Tere Abelleira: «Por fin hablamos de fútbol»
Descripción:España se conjura para un objetivo que empezó en Oliva, continúa en estos dos partidos de la Nations League y desemboca en París, concentradas las jugadoras de Montse Tomé en alcanzar esos Juegos Olímpicos que nunca se han disfrutado antes. Tampoco se había ganado un Mundial y se cumplen dos meses y una semana desde aquel hito. Lo quieren celebrar en el campo, ante Italia, aunque no sea un rival fácil porque han sido dos triunfos en una decena de enfrentamientos disputados a domicilio.Pero en el Estadio Arechi de Salerno se junta un equipo que ha recuperado a Jenni, la alegría y la calma, sobre todo la calma, dedicadas las jugadoras en el fútbol, intentando ya olvidar todo lo que las ha rodeado desde aquel 20 de agosto en el Estadio de Sídney.Porque, conseguida la estrellas en el pecho que lucen con or

DEBUG:scrapy.downloadermiddlewares.redirect:Redirecting (301) to <GET https://www.laverdad.es/rss/2.0/portada/> from <GET https://www.laverdad.es/rss/2.0/portada>
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.europapress.es/rss/rss.aspx> (referer: None)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://feeds.elpais.com/mrss-s/pages/ep/site/elpais.com/portada> (referer: None)


-------------------------
URL:https://www.europapress.es/internacional/noticia-guerra-israel-hamas-gaza-directo-ultimas-noticias-hoy-20231018074949.html
Título:Guerra Israel - Gaza | Directo: Hamás asegura que al menos 50 rehenes han muerto por los bombardeos de Israel
Descripción:Sigue toda la actualidad de Europa Press a través de nuestro nuevo canal de WhatsApp
Fecha:Wed, 18 Oct 2023 05:49:49 GMT
-------------------------
-------------------------
URL:https://www.europapress.es/nacional/noticia-sanchez-propone-conferencia-paz-seis-meses-cree-solo-falta-reconocer-estado-palestino-20231026153809.html
Título:Sánchez propone una conferencia de paz "en seis meses" y cree que solo falta por reconocer al Estado palestino
Descripción:El presidente del Gobierno en funciones, Pedro Sánchez, ha propuesto celebrar una conferencia de paz para Oriente Próximo "en seis meses". Además, ha insistido en la solución de dos Estados, aunque ha precisado que el único que falta por reconocer es el pueblo 

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.laverdad.es/rss/2.0/portada/> (referer: None)


-------------------------
URL:https://elpais.com/eps/2023-10-26/la-trampa-de-la-multitarea.html
Título:La multitarea es tarea imposible
Descripción:La ciencia ha probado que somos incapaces de prestar atención a varias cosas al mismo tiempo. Intentarlo tiene importantes efectos negativos en la salud mental y la eficacia
Fecha:Thu, 26 Oct 2023 03:40:00 GMT
-------------------------
-------------------------
URL:https://elpais.com/eps/2023-10-24/tiktok-el-nuevo-escaparate-para-los-musicos-emergentes.html
Título:TikTok, el nuevo escaparate para los músicos emergentes
Descripción:Las discográficas buscan dentro de la red social a nuevos artistas que intentan abrirse camino en la industria con la publicación de vídeos cortos cantando
Fecha:Tue, 24 Oct 2023 03:40:00 GMT
-------------------------
-------------------------
URL:https://elpais.com/smoda/famosos/britney-spears-explica-por-que-le-gusta-posar-desnuda-en-instagram.html
Título:Britney Spears explica por qué le gusta posar desnuda en 

INFO:scrapy.core.engine:Closing spider (finished)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 1998,
 'downloader/request_count': 5,
 'downloader/request_method_count/GET': 5,
 'downloader/response_bytes': 227217,
 'downloader/response_count': 5,
 'downloader/response_status_count/200': 4,
 'downloader/response_status_count/301': 1,
 'elapsed_time_seconds': 3.272127,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 10, 26, 16, 35, 38, 235650, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 659578,
 'httpcompression/response_count': 4,
 'log_count/DEBUG': 6,
 'log_count/INFO': 10,
 'memusage/max': 170262528,
 'memusage/startup': 170262528,
 'response_received_count': 4,
 'scheduler/dequeued': 5,
 'scheduler/dequeued/memory': 5,
 'scheduler/enqueued': 5,
 'scheduler/enqueued/memory': 5,
 'start_time': datetime.datetime(2023, 10, 26, 16, 35, 34, 963523, tzinfo=datetime.timezone.utc)}
INFO:scrapy.core.engine:Spider cl


URL:https://www.laverdad.es/sorteos/once/once-comprobar-resultados-sorteo-miercoles-octubre-2023-20231025213633-nt.html
Título:ONCE: Comprobar resultados del sorteo del miércoles 25 de octubre de 2023
Descripción:En el sorteo del cupón diario de la ONCE de hoy miércoles 25 de octubre de 2023, ha resultado premiado el número 50120, serie 30
Fecha:Wed, 25 Oct 2023 21:36:34 +0200
-------------------------
-------------------------
URL:https://www.laverdad.es/murcia/cultura-destina-300000-euros-atraer-grandes-rodajes-20231025120515-nt.html
Título:300.000 euros para atraer grandes rodajes a la Región
Descripción:La convocatoria de Cultura establece tres ayudas a películas o series de productoras de fuera, que deberán contratar a profesionales murcianos
Fecha:Wed, 25 Oct 2023 20:47:06 +0200
-------------------------
-------------------------
URL:https://www.laverdad.es/pantallas/series/septima-entrega-elite-aburrida-floja-20231025204436-ntrc.html
Título:La séptima entrega de 'Élite' es la m